In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
!pip install keras
!pip install sklearn

In [0]:
import pandas as pd
import numpy as np

import io
import os

import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation, Bidirectional
from keras.layers import Embedding

import sklearn
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


Using TensorFlow backend.


In [0]:
#Data collection -- read from Drive
data_frame = pd.read_csv('/content/drive/Shared drives/Eye Detection LSTM/eyeDetection.csv')
#print("DF Shape before shuffle",data_frame.shape)
#shuffle the data
#data_frame = data_frame.sample(frac=1)
#print("DF Shape after shuffle",data_frame.shape)

#defining train and test data set size --- 67% for training and 33% for testing --- of total 14980
#X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.33) --- Alternative ***** Try later__selfMemo ***** 

train_size = int(len(data_frame)*0.7)
print("Train_size",train_size)
#train_size --- 10036 
test_size = len(data_frame)-train_size
print("Test_size",test_size)
#test_size --- 4944

# #Spliting data frame into train and test
train_df = data_frame.loc[:train_size]
# # print(train_df.head())
test_df=data_frame.loc[train_size:]
# # print(test_df.head())

# #Splitting Dataset and Labels and converting them to Numpy arrays for processing
# #X_train -- train data set  --- Y_train --- train labels

X_train=train_df.iloc[:,:14].to_numpy()
Y_train=train_df.iloc[:,14].to_numpy()

#X_test  -- test data set   --- Y_test  --- test labels
X_test=test_df.iloc[:,:14].to_numpy()
Y_test=test_df.iloc[:,14:].to_numpy()
X_test
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)
print(X_test[0,0])

#X = data_frame.iloc[:, :14]
#Y = data_frame.iloc[:, 14]
#print("Data Frame Shape",data_frame.shape)
#print("X Shape",X.shape)
#print("Y Shape",Y.shape)
#print(X.head())
#print(Y.head())

Train_size 10486
Test_size 4494
(10487, 14)
(10487,)
(4494, 14)
(4494, 1)
4303.59


In [0]:
#Normalizing the data using Standard Scaler
# X_train=StandardScaler().fit_transform(X_train)
# X_test=StandardScaler().fit_transform(X_test)

#OneHotEncoding the labels
# ohe = OneHotEncoder()
# Y_train = OneHotEncoder().fit_transform(Y_train).toarray()
# Y_test=OneHotEncoder().fit_transform(Y_test).toarray()
# Y_test


In [0]:

#reshaping the input data for LSTM only

# X_train=X_train.reshape(X_train.shape[0],1,X_train.shape[1])
# X_test=X_test.reshape(X_test.shape[0],1,X_test.shape[1])

In [0]:
###LSTM
model = Sequential()
model.add(Embedding(20000, 128))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='relu'))

In [0]:
###LSTM
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [0]:
history = model.fit(X, Y,validation_split = 0.1, epochs=20, batch_size=64, shuffle=True)

Train on 13482 samples, validate on 1498 samples
Epoch 1/20
13482/13482 [==============================] - 15s 1ms/step - loss: 0.2411 - acc: 0.5909 - val_loss: 0.2185 - val_acc: 0.6549
Epoch 2/20
13482/13482 [==============================] - 13s 990us/step - loss: 0.2051 - acc: 0.6799 - val_loss: 0.2100 - val_acc: 0.6736
Epoch 3/20
13482/13482 [==============================] - 13s 975us/step - loss: 0.1913 - acc: 0.7089 - val_loss: 0.2051 - val_acc: 0.6869
Epoch 4/20
13482/13482 [==============================] - 13s 967us/step - loss: 0.1831 - acc: 0.7243 - val_loss: 0.1979 - val_acc: 0.6949
Epoch 5/20
13482/13482 [==============================] - 13s 973us/step - loss: 0.1759 - acc: 0.7394 - val_loss: 0.1963 - val_acc: 0.7029
Epoch 6/20
13482/13482 [==============================] - 13s 982us/step - loss: 0.1677 - acc: 0.7539 - val_loss: 0.1859 - val_acc: 0.7196
Epoch 7/20
13482/13482 [==============================] - 13s 971us/step - loss: 0.1591 - acc: 0.7687 - val_loss: 0.177

In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               263168    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 2,823,425
Trainable params: 2,823,425
Non-trainable params: 0
_________________________________________________________________


In [0]:
#Test 
Y_pred=model.predict(X)

#inverse OHE
pred=list()
for i in range(len(Y_pred)):
  pred.append(np.argmax(Y_pred[i]))
test=list()
for i in range(len(Y_test)):
  test.append(np.argmax(Y_test[i]))
  



In [0]:
pred

In [0]:
a=accuracy_score(pred,test)
a

ValueError: ignored